In [1]:
import pandas as pd
import numpy as np
import csv

# União dos atributos de localidade para dataset Doenças

In [2]:
def preProcessamento(dataset):
    dataset = doencas.copy() 
    local = []
    cidades = doencas.cidade
    estados = doencas.estado

    for i in range(0,len(cidades)):
        local.append(str('{}, {}'.format(cidades[i], estados[i])))
        
    pd.DataFrame(local)
    dataset = dataset.drop("cidade",axis=1)
    dataset = dataset.drop("estado",axis=1)

    dataset['local'] = local

    sequence = ['id', 'data', 'genero', 'local', 'doenca']
    dataset = dataset.reindex(columns=sequence)
    
    return dataset

# União dos atributos de localidade para dataset Background

In [3]:
def preProcessamentoBack(dataset):
    dataset = background.copy() 
    local = []
    cidades = background.cidade
    estados = background.estado

    for i in range(0,len(cidades)):
        local.append(str('{}, {}'.format(cidades[i], estados[i])))
        
    pd.DataFrame(local)
    dataset = dataset.drop("cidade",axis=1)
    dataset = dataset.drop("estado",axis=1)

    dataset['local'] = local

    sequence = ['nome', 'data', 'genero', 'local']
    dataset = dataset.reindex(columns=sequence)
    
    return dataset

# Geração do reticulado de generalizações

In [4]:
def gerarHierarquias(NivelMax):
    hierarquias = [NivelMax]
    while([0,0,0] not in hierarquias):
        for i in range(0, len(hierarquias)):
            for j in range(0, len(NivelMax)):
                h = hierarquias[i].copy()
                h[j] = h[j] - 1
                if(h[j]>=0):
                    if(h not in hierarquias):
                        hierarquias.append(h)
    return hierarquias

# Buscar nível na árvore de generalização

In [5]:
def buscaNivel(elemento):
    return elemento.count('*')

# Função de precisão

In [6]:
def precisao(dataset, N, D):
    somatorio = 0
    for i in range(0,N):
        for j in range(0,D):
            if(i == 0):                
                h = buscaNivel(dataset[j][i])
                HGV = 3
            elif(i == 1):
                h = buscaNivel(dataset[j][i])
                HGV = 1
            else:
                h = buscaNivel(dataset[j][i])
                HGV = 2
            somatorio += h/HGV
    resultado = 1 - (somatorio)/(D*N)
    return resultado

# Verificar se dataset atende o delta-presença

In [7]:
def verificarDelta(dataset, datasetBackground, delta):
    regDataset = []
    contRegDataset = []
    regBack = []
    contRegBack = []
    for i in dataset:
        reg = [i[1],i[2], i[3]]
        cont = 1
        if(reg not in regDataset):
            regDataset.append(reg)
            contRegDataset.append(cont)
        else:
            for j in range(0, len(regDataset)):
                if(reg == regDataset[j]):
                    contRegDataset[j] = contRegDataset[j] + 1
    for i in datasetBackground:
        reg = [i[1],i[2], i[3]]
        cont = 1
        if(reg not in regBack):
            regBack.append(reg)
            contRegBack.append(cont)
        else:
            for j in range(0, len(regBack)):
                if(reg == regBack[j]):
                    contRegBack[j] = contRegBack[j] + 1
    for i in range(0, len(regBack)):
        if(regDataset[0] == regBack[i]):
            probMin = contRegDataset[0]/contRegBack[i]
            probMax = contRegDataset[0]/contRegBack[i]
            
    for i in range(0, len(contRegDataset)):
        for j in range(0, len(contRegBack)):
            if(regDataset[i] == regBack[j]):
                prob = contRegDataset[i]/contRegBack[j]
                if(prob <= probMin):
                    probMin = prob
                elif(prob > probMax):
                    probMax = prob
    if(probMin>=delta[0] and probMax<=delta[1]):
        return True
    else:
        return False

# Selecionar dataset válido com maior utilidade

In [8]:
def verificarPrecisao(datasets):
    precisoes = []
    for dataset in datasets:
        precisoes.append(precisao(dataset, 3, len(dataset)))
    precisoes = np.array(precisoes)
    return datasets[np.argmax(precisoes)]

# Delta-presença

In [9]:
def deltaPresenca(dataset, datasetBackground, delta, hierarquias):
    dataset = np.array(dataset)
    datasetBack = np.array(datasetBackground)
    datasets = []
    for h in hierarquias:
        dado = np.copy(dataset)
        dadoBack = np.copy(datasetBack)
        for i in dado:
            data = np.array(i[1].split("/"))
            local = np.array(i[3].split(","))
            if(h[0] == 3):
                i[1] = '*/*/*'
            elif(h[0] == 2):
                i[1] = '*/*/'+data[2]
            elif(h[0] == 1):
                i[1] = '*/'+data[1]+'/'+data[2]
            if(h[1] == 1):
                i[2] = "*"
            if(h[2] == 2):
                i[3] = "*,*"
            elif(h[2] == 1):
                i[3] = "*,"+local[1]
        for i in dadoBack:
            data = np.array(i[1].split("/"))
            local = np.array(i[3].split(","))
            if(h[0] == 3):
                i[1] = '*/*/*'
            elif(h[0] == 2):
                i[1] = '*/*/'+data[2]
            elif(h[0] == 1):
                i[1] = '*/'+data[1]+'/'+data[2]
            if(h[1] == 1):
                i[2] = "*"
            if(h[2] == 2):
                i[3] = "*,*"
            elif(h[2] == 1):
                i[3] = "*,"+local[1]
        dadoValido = verificarDelta(dado, dadoBack, delta)
        
        if(dadoValido):
            df = pd.DataFrame(dado, columns=['id','data','genero','local','doenca'])
            df = df.drop('id', axis=1)
            df = np.array(df)
            datasets.append(df)
            
    datasetFinal = verificarPrecisao(datasets)
    
    return datasetFinal

# Organizar dataset e gerar CSV

In [10]:
def posProcessamento(dataset, delta):
    regUnicos = []
    valoresDoencas = []
    for i in dataset:
        reg = [i[0], i[1], i[2]]
        if(reg not in regUnicos):
            regUnicos.append(reg)
    for reg in regUnicos:
        doencas = []
        for i in dataset:
            registro = [i[0], i[1], i[2]]
            if(registro == reg):
                doencas.append(i[3])
        valoresDoencas.append(doencas)
    datasetFinal = []
    for i in range(0, len(regUnicos)):
        for j in valoresDoencas[i]:
            registro = regUnicos[i][0], regUnicos[i][1], regUnicos[i][2], j
            datasetFinal.append(registro)
    
    datasetFinal = pd.DataFrame(datasetFinal, columns=['data','genero','local','doenca'])
    arquivo = 'doencas_'+str(delta)+'.csv'
    datasetFinal.to_csv(arquivo, index=False)
    datasetFinal = np.array(datasetFinal)
    return datasetFinal

# Execução

In [11]:
doencas = pd.read_csv('doencas.csv')
background = pd.read_csv('background.csv')

In [12]:
dfDoencas = preProcessamento(doencas)
dfBackground = preProcessamentoBack(background)

In [13]:
NivelMax = ([3, 1, 2])
hierarquias = [NivelMax]
hierarquias = gerarHierarquias(NivelMax)

In [14]:
deltas = [[0.1, 0.5], [0.1, 0.4], [0.1, 0.3], [0.1, 0.2]]
porcentagens = [40, 30, 20, 10]
for i in range(0, len(deltas)):
    datasetFinal = deltaPresenca(dfDoencas, dfBackground, deltas[i], hierarquias)
    datasetFinal = posProcessamento(datasetFinal, porcentagens[i])
    print(precisao(datasetFinal, 3, datasetFinal.shape[0]))

0.611111111111112
0.611111111111112
0.611111111111112
0.0
